In [1]:
import os
import whisper
import openai
import requests_html
import dataset
import pprint

from dotenv import load_dotenv
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

In [ ]:
navegador = Chrome(service=Service(ChromeDriverManager().install()))s

In [ ]:
navegador.maximize_window()

In [ ]:
navegador.get("https://pyvideo.org/index.html")

In [ ]:
links = navegador.find_elements(By.XPATH, '//article/a')

In [ ]:
for link in links:
    print(link.get_attribute('href'))

In [ ]:
links[0].click()

In [ ]:
links_video = navegador.find_elements(By.LINK_TEXT, 'ARCHIVE')

In [ ]:
for link_video in links_video:
    print(link_video.get_attribute('href'))

In [ ]:
video_url = links_video[1].get_attribute('href')
video_url

In [ ]:
sessao = requests_html.HTMLSession()

In [ ]:
resposta = sessao.get(video_url)

In [ ]:
video_download = resposta.html.xpath('//a[contains(text(), "MPEG4")]/@href', first=True)
video_download

In [ ]:
resposta = sessao.get(f'https://archive.org{video_download}')
with open('video.mp4', 'wb') as arquivo:
    arquivo.write(resposta.content)

In [ ]:
modelo = whisper.load_model("small")

In [ ]:
ffmpeg_extract_subclip("video.mp4", 0, 120, targetname="video_cortado.mp4")

In [ ]:
transcricao = modelo.transcribe('video_cortado.mp4')

In [ ]:
pprint.pprint(transcricao)

In [ ]:
pprint.pprint(transcricao['text'])

In [ ]:
load_dotenv()

In [ ]:
openai.api_key = os.environ['OPENAI_KEY']

In [ ]:
mensagens = []
mensagens.append(
    {"role": "user", "content": f"traduza o seguinte de inglês para português: {transcricao['text']}"},
)
chat = openai.ChatCompletion.create(
    model="gpt-3.5-turbo", messages=mensagens
)
chat['choices'][0]['message']['content']